<a href="https://colab.research.google.com/github/edgaracabral/Kaggle_BankMarketing/blob/main/02_Pre_Processamento_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Bank Marketing Case
## Pré-Processamento de Dados

Neste fase faremos o pré-processamento dos dados do Kaggle Bank Marketing Case:
* Codificar, normalizar, padronizar variáveis (justificar escolhas)
* Separar em bases de treino/teste

# **Introdução**

## 1) Preparação dos Dados (DataPrep)
- Gerar Metadados da ABT (Tabela Analítica de Modelagem)
- Tratamento de missing (nulos)
 - Média para variáveis numéricas
 - 'Desconhecido' para variáveis categóricas
- Tratamento de categóricas de alta cardinalidade (LabelEncoder)
- Tratamento de categóricas de baixa cardinalidade (OneHotEncoder)
- Aplicar normalização a toda tabela de modelagem (ABT)
- Gerar artefatos para implantação do data prep realizado

## Carregando Google Drive e bibliotecas utilizadas no projeto

In [149]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


`pod_academy_functions` é a nossa biblioteca criada no curso de DS

In [150]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

# Redirecione para o folder on pod_academy_functions.py is localizado
%cd /content/drive/MyDrive/PoD Academy/modelos/Hackaton_DS_2023/PoD Framework
import pod_academy_functions as pod

/content/drive/MyDrive/PoD Academy/modelos/Hackaton_DS_2023/PoD Framework


## Lendo os dados disponibilizados

In [151]:
# Armazene o caminho do folder de Dados em file_path
file_path = '/content/drive/MyDrive/Kaggle/Bank\ Marketing/01\ original\ data'

# Vá para o folder de dados
%cd $file_path

/content/drive/MyDrive/Kaggle/Bank Marketing/01 original data


In [152]:
# Carregar o arquivo CSV em um DataFrame
df_train_00 = pd.read_csv('bank-full.csv', sep=';')

# Exibir o tamanho do arquivo lido
print(f"df_train_00.shape = {df_train_00.shape}")

# Exibir as primeiras linhas do DataFrame
df_train_00.head()

df_train_00.shape = (45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [153]:
# Checar Frequencia nas variáveis categóricas
print(pod.pod_custom_lowfreq(df_train_00))

      Coluna     Indice  Frequencia
7      month        dec         214
0        job    unknown         288
3    default        yes         815
8   poutcome    success        1511
2  education    unknown        1857
6    contact  telephone        2906
1    marital   divorced        5207
9          y        yes        5289
5       loan        yes        7244
4    housing         no       20081


Redirecionado para pasta `dataprep`

In [154]:
# Adicionado a coluna 'id' ao dataframe, para termos uma chave primária para cada linha da tabela
df_train_00['id'] = range(1, len(df_train_00) + 1)

In [155]:
# A partir de agora, todos os artefatos gerados serão colocados dentro da pasta dataprep
%cd /content/drive/MyDrive/Kaggle/Bank\ Marketing/02\ data\ prep

/content/drive/MyDrive/Kaggle/Bank Marketing/02 data prep


## Renomeando ID e Target do original para o padrão do Framework

In [156]:
# Renomeie as colunas de id e target para o padrão do Framework (id, target)
#df_train_00.rename(columns={'id': 'id', 'y': 'target'}, inplace=True)

# Crie a coluna target a partir da coluna y
df_train_00['target'] = df_train_00['y'].map({'yes': 1, 'no': 0})

# Remover a coluna y da base de treino
df_train_00 = df_train_00.drop(axis=1, columns=['y'])

## Amostragem da base principal
- Serve também para testar se OneHot Encoding e Label Encoding estão tratando excessões

In [157]:
#df_train_00_sample = df_train_00.sample(n=10000, random_state=42)
df_train_00_sample = df_train_00.copy()
df_train_00_sample.shape

(45211, 18)

In [158]:
df_train_00_sample

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,id,target
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,2,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,3,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,4,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,45207,1
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,45208,1
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,45209,1
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,45210,0


## Separando dados para garantir validação cruzada Holdout 70/30

In [159]:
# Suponha que você queira separar 70% dos dados para treino e 30% para validação
train, test = train_test_split(df_train_00_sample, test_size=0.3, random_state=42)
train.shape,test.shape

((31647, 18), (13564, 18))

In [160]:
train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,id,target
10747,36,technician,single,tertiary,no,0,no,no,unknown,17,jun,153,4,-1,0,unknown,10748,0
26054,56,entrepreneur,married,secondary,no,196,no,no,cellular,19,nov,312,3,-1,0,unknown,26055,0
9125,46,blue-collar,married,secondary,no,0,yes,no,unknown,5,jun,83,2,-1,0,unknown,9126,0
41659,41,management,divorced,tertiary,no,3426,no,no,cellular,1,oct,302,1,119,5,success,41660,0
4443,38,blue-collar,married,secondary,no,0,yes,no,unknown,20,may,90,1,-1,0,unknown,4444,0


In [161]:
# Criando um novo dataframe baseado no original (Original fazia transformações aqui)
df_train_01 = train.copy()
df_test_01 = test.copy()

In [162]:
df_train_01.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,id,target
10747,36,technician,single,tertiary,no,0,no,no,unknown,17,jun,153,4,-1,0,unknown,10748,0
26054,56,entrepreneur,married,secondary,no,196,no,no,cellular,19,nov,312,3,-1,0,unknown,26055,0
9125,46,blue-collar,married,secondary,no,0,yes,no,unknown,5,jun,83,2,-1,0,unknown,9126,0
41659,41,management,divorced,tertiary,no,3426,no,no,cellular,1,oct,302,1,119,5,success,41660,0
4443,38,blue-collar,married,secondary,no,0,yes,no,unknown,20,may,90,1,-1,0,unknown,4444,0


In [163]:
metadados = pod.pod_academy_generate_metadata(df_train_01,
                                          ids=['id'],
                                          targets=['target'],
                                          orderby = 'PC_NULOS')

metadados

,FEATURE,USO_FEATURE,QT_NULOS,PC_NULOS,CARDINALIDADE,TIPO_FEATURE
0,age,Explicativa,0,0.0,77,int64
1,job,Explicativa,0,0.0,12,object
2,id,ID,0,0.0,31647,int64
3,poutcome,Explicativa,0,0.0,4,object
4,previous,Explicativa,0,0.0,39,int64
5,pdays,Explicativa,0,0.0,518,int64
6,campaign,Explicativa,0,0.0,47,int64
7,duration,Explicativa,0,0.0,1455,int64
8,month,Explicativa,0,0.0,12,object
9,day,Explicativa,0,0.0,31,int64


## Excluindo variáveis com mais que 70% de nulos

In [164]:
missing_cutoff = 70

drop_vars_nulos = metadados[(metadados['PC_NULOS'] >= missing_cutoff)]
lista_drop_vars = list(drop_vars_nulos.FEATURE.values)

print('Variáveis que serão excluídas por alto percentual de nulos: ',lista_drop_vars)
# retirando lista de variáveis com alto percentual de nulos
df_train_02 = df_train_01.drop(axis=1,columns=lista_drop_vars)
df_train_02.shape

Variáveis que serão excluídas por alto percentual de nulos:  []


(31647, 18)

In [165]:
# Salvar a lista em um arquivo .pkl
with open('prd_drop_nullvars_hktn.pkl', 'wb') as f:
    pickle.dump(lista_drop_vars, f)

Tratando a base de teste da mesma forma que a de treino

In [166]:
# retirando lista de variáveis com alto percentual de nulos
df_test_02 = df_test_01.drop(axis=1,columns=lista_drop_vars)
df_test_02.shape

(13564, 18)

In [167]:
df_train_02.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,id,target
10747,36,technician,single,tertiary,no,0,no,no,unknown,17,jun,153,4,-1,0,unknown,10748,0
26054,56,entrepreneur,married,secondary,no,196,no,no,cellular,19,nov,312,3,-1,0,unknown,26055,0
9125,46,blue-collar,married,secondary,no,0,yes,no,unknown,5,jun,83,2,-1,0,unknown,9126,0
41659,41,management,divorced,tertiary,no,3426,no,no,cellular,1,oct,302,1,119,5,success,41660,0
4443,38,blue-collar,married,secondary,no,0,yes,no,unknown,20,may,90,1,-1,0,unknown,4444,0


In [168]:
# Retirar ID e Target do tratamento de nulos na base de treino
df_train_02 = df_train_02.drop(axis=1, columns=['id', 'target'])
df_train_02.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
10747,36,technician,single,tertiary,no,0,no,no,unknown,17,jun,153,4,-1,0,unknown
26054,56,entrepreneur,married,secondary,no,196,no,no,cellular,19,nov,312,3,-1,0,unknown
9125,46,blue-collar,married,secondary,no,0,yes,no,unknown,5,jun,83,2,-1,0,unknown
41659,41,management,divorced,tertiary,no,3426,no,no,cellular,1,oct,302,1,119,5,success
4443,38,blue-collar,married,secondary,no,0,yes,no,unknown,20,may,90,1,-1,0,unknown


In [169]:
# Retirar ID e Target do tratamento de nulos na base de teste
df_test_02 = df_test_02.drop(axis=1, columns=['id', 'target'])
df_test_02.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
3776,40,blue-collar,married,secondary,no,580,yes,no,unknown,16,may,192,1,-1,0,unknown
9928,47,services,single,secondary,no,3644,no,no,unknown,9,jun,83,2,-1,0,unknown
33409,25,student,single,tertiary,no,538,yes,no,cellular,20,apr,226,1,-1,0,unknown
31885,42,management,married,tertiary,no,1773,no,no,cellular,9,apr,311,1,336,1,failure
15738,56,management,married,tertiary,no,217,no,yes,cellular,21,jul,121,2,-1,0,unknown


## Substituindo os nulos
- pela média para variáveis numéricas
- por 'POD_VERIFICAR" para categóricas

In [170]:
# Substituindo os nulos na base de treino
df_train_03, means = pod.pod_custom_fillna(df_train_02)

with open('prd_fillna_hktn.pkl', 'wb') as f:
  pickle.dump(means, f)

In [171]:
with open('prd_fillna_hktn.pkl', 'rb') as f:
  loaded_means = pickle.load(f)
loaded_means

{'age': 40.941669036559546,
 'balance': 1359.3179719129555,
 'day': 15.829620501153348,
 'duration': 257.57006983284356,
 'campaign': 2.772237494865232,
 'pdays': 225.2088520055325,
 'previous': 0.585869118715834}

In [172]:
# Substituindo os nulos na base de teste usando as médias da base de treino
df_test_03 = pod.pod_custom_fillna_prod(df_test_02, loaded_means)
df_test_03.shape

(13564, 16)

In [173]:
df_test_03.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
3776,40,blue-collar,married,secondary,no,580.0,yes,no,unknown,16,may,192,1,225.208852,0,unknown
9928,47,services,single,secondary,no,3644.0,no,no,unknown,9,jun,83,2,225.208852,0,unknown
33409,25,student,single,tertiary,no,538.0,yes,no,cellular,20,apr,226,1,225.208852,0,unknown
31885,42,management,married,tertiary,no,1773.0,no,no,cellular,9,apr,311,1,336.000000,1,failure
15738,56,management,married,tertiary,no,217.0,no,yes,cellular,21,jul,121,2,225.208852,0,unknown


## Tratamento de categóricas de alta cardinalidade (LabelEncoder)

In [174]:
# Identifica colunas para LabelEncoder que não estejam na lista lista_drop_vars
card_cutoff = 35
df_categ_labelenc = metadados[(metadados['CARDINALIDADE'] > card_cutoff) & (metadados['TIPO_FEATURE'] == 'object')]
lista_vars_abt = list(df_train_03.columns)
lista_lenc = list(df_categ_labelenc.FEATURE.values)

for item in lista_drop_vars:
    if item in lista_lenc:
        lista_lenc.remove(item)

print('Lista de vars para Label Encoding: ',lista_lenc)

Lista de vars para Label Encoding:  []


In [175]:
# transforma via LabelEncoder colunas na base de treino
import pickle
from sklearn.preprocessing import LabelEncoder

encoders = {}

for col in lista_lenc:
    encoder = LabelEncoder()
    df_train_03[col] = encoder.fit_transform(df_train_03[col])

    # Armazena o encoder para a coluna atual em um dicionário
    encoders[col] = encoder

# Salva o dicionário de encoders e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoders': encoders,
    'columns': lista_lenc
}

with open('prd_labelenc_hktn.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)

In [176]:
print(data_to_serialize)

{'encoders': {}, 'columns': []}


In [177]:
# Carregar os encoders e a lista de colunas
with open('prd_labelenc_hktn.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

# Suponha df_test_03 como sua base de teste
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        df_test_03[col] = loaded_encoders[col].transform(df_test_03[col])


## Tratamento para categóricas de baixa cardinalidade

In [178]:
print(df_train_03.shape)
print(df_test_03.shape)

(31647, 16)
(13564, 16)


In [179]:
import pickle
from sklearn.preprocessing import OneHotEncoder

card_cutoff = 35
df_categ_onehot = metadados[(metadados['CARDINALIDADE'] <= card_cutoff) & (metadados['TIPO_FEATURE'] == 'object')]
lista_onehot = list(df_categ_onehot.FEATURE.values)
print('Lista de vars para OneHot Encoding: ',lista_onehot)

# Instanciando o encoder
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Aplicando o one-hot encoding
encoded_data = encoder.fit_transform(df_train_03[lista_onehot])
encoded_cols = encoder.get_feature_names_out(lista_onehot)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=df_train_03.index)

df_train_03 = pd.concat([df_train_03.drop(lista_onehot, axis=1), encoded_df], axis=1)

# Salva o encoder e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoder': encoder,
    'columns': lista_onehot
}

with open('prd_onehotenc_hktn.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)

df_train_03.shape

Lista de vars para OneHot Encoding:  ['job', 'poutcome', 'month', 'contact', 'loan', 'housing', 'default', 'education', 'marital']


(31647, 42)

In [180]:
# Carregar o encoder e a lista de colunas
with open('prd_onehotenc_hktn.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoder = loaded_data['encoder']
loaded_columns = loaded_data['columns']

# Suponha df_test_03 como sua base de teste
encoded_data_test = loaded_encoder.transform(df_test_03[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=df_test_03.index)

df_test_03 = pd.concat([df_test_03.drop(loaded_columns, axis=1), encoded_df_test], axis=1)

df_test_03.shape

(13564, 42)

## Aplicar padronização a toda tabela de modelagem tratada ate este ponto

In [181]:
import pickle
from sklearn.preprocessing import StandardScaler

# Excluindo IDs e Targets
df_id_target = metadados[(metadados['USO_FEATURE'] == 'ID') | (metadados['USO_FEATURE'] == 'Target')]
lista_id_target = list(df_id_target.FEATURE.values)
print('Lista de IDs e Target: ',lista_id_target)

# Instanciando o scaler
scaler = StandardScaler()

# Selecionando colunas numéricas
numeric_cols = df_train_03.select_dtypes(include=['float64', 'int64','int32']).columns

# Aplicando a normalização
df_train_03[numeric_cols] = scaler.fit_transform(df_train_03[numeric_cols])

# Salva o scaler em um arquivo .pkl
with open('prd_scaler_hktn.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print(scaler)
df_train_03.shape

Lista de IDs e Target:  ['id', 'target']
StandardScaler()


(31647, 42)

In [182]:
# Carregar o scaler
with open('prd_scaler_hktn.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# Suponha df_test_03 como sua base de teste

# Selecionando colunas numéricas
numeric_cols = df_test_03.select_dtypes(include=['float64', 'int64','int32']).columns

# Aplicando a normalização
df_test_03[numeric_cols] = loaded_scaler.fit_transform(df_test_03[numeric_cols])

df_test_03.shape

(13564, 42)

In [183]:
df_test_03.head()

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,contact_telephone,contact_unknown,loan_yes,housing_yes,default_yes,education_secondary,education_tertiary,education_unknown,marital_married,marital_single
3776,-0.087221,-0.248404,0.029771,-0.265462,-0.588636,0.008043,-0.289532,1.930746,-0.185029,-0.166294,...,-0.261634,1.558712,-0.435928,0.894768,-0.122953,0.988419,-0.660382,-0.202638,0.819560,-0.629323
9928,0.573919,0.709957,-0.811520,-0.693838,-0.251164,0.008043,-0.289532,-0.517935,-0.185029,-0.166294,...,-0.261634,1.558712,-0.435928,-1.117608,-0.122953,0.988419,-0.660382,-0.202638,-1.220167,1.589009
33409,-1.503949,-0.261541,0.510510,-0.131840,-0.588636,0.008043,-0.289532,-0.517935,-0.185029,-0.166294,...,-0.261634,-0.641555,-0.435928,0.894768,-0.122953,-1.011717,1.514276,-0.202638,-1.220167,1.589009
31885,0.101676,0.124744,-0.811520,0.202214,-0.588636,2.327377,0.220761,-0.517935,-0.185029,-0.166294,...,-0.261634,-0.641555,-0.435928,-1.117608,-0.122953,-1.011717,1.514276,-0.202638,0.819560,-0.629323
15738,1.423956,-0.361943,0.630694,-0.544496,-0.251164,0.008043,-0.289532,-0.517935,-0.185029,-0.166294,...,-0.261634,-0.641555,2.293956,-1.117608,-0.122953,-1.011717,1.514276,-0.202638,0.819560,-0.629323


In [184]:
## Trazer o id e target para a tabela pós dataprep

abt_train = df_train_03.merge(train[['id','target']], left_index=True, right_index=True, how='inner')
abt_test = df_test_03.merge(test[['id','target']], left_index=True, right_index=True, how='inner')

print(abt_train.shape)
print(abt_test.shape)

(31647, 44)
(13564, 44)


In [185]:
abt_train.head()

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,loan_yes,housing_yes,default_yes,education_secondary,education_tertiary,education_unknown,marital_married,marital_single,id,target
10747,-0.464799,-0.456680,0.140619,-0.404011,0.389277,-1.138504e-15,-0.240512,-0.526225,-0.184151,-0.168627,...,-0.437178,-1.119132,-0.140544,-1.033257,1.564172,-0.208793,-1.233806,1.593468,10748,0
26054,1.416343,-0.390831,0.380915,0.210292,0.072215,-1.138504e-15,-0.240512,-0.526225,5.430326,-0.168627,...,-0.437178,-1.119132,-0.140544,0.967813,-0.639316,-0.208793,0.810500,-0.627562,26055,0
9125,0.475772,-0.456680,-1.301157,-0.674459,-0.244847,-1.138504e-15,-0.240512,1.900329,-0.184151,-0.168627,...,-0.437178,0.893550,-0.140544,0.967813,-0.639316,-0.208793,0.810500,-0.627562,9126,0
41659,0.005486,0.694328,-1.781749,0.171657,-0.561910,-2.127233e+00,1.812098,-0.526225,-0.184151,-0.168627,...,-0.437178,-1.119132,-0.140544,-1.033257,1.564172,-0.208793,-1.233806,-0.627562,41660,0
4443,-0.276685,-0.456680,0.501063,-0.647414,-0.561910,-1.138504e-15,-0.240512,1.900329,-0.184151,-0.168627,...,-0.437178,0.893550,-0.140544,0.967813,-0.639316,-0.208793,0.810500,-0.627562,4444,0


## Salvando tabelas de treino e teste pós preparação dos dados

In [186]:
abt_train.to_csv('abt_train.csv')
abt_test.to_csv('abt_test.csv')